# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final project: Machine Learning** </center>
---

**Date**: November, 2025

**Student Name**: Díaz Campos José Juan

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from jjodiaz.spark_utils import SparkUtils
from pyspark.sql.functions import col

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler

spark = SparkSession.builder \
    .appName("ML_FinalProject_JoseJuanDiaz") \
    .master("local[*]") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

print("SparkSession iniciada y configurada.")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/11 04:38:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/11 04:38:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


SparkSession iniciada y configurada.


## Machine Learning algorithm to use (10 points)

### Problema a Resolver: Clustering
Para este proyecto, el problema a resolver es la **segmentación de clientes** (Customer Segmentation), que es una tarea de **Clustering** (Aprendizaje No Supervisado). El algoritmo seleccionado de la lista provista es **K-Means**
### Justificación de la Elección

He seleccionado este problema por su alta relevancia práctica en el análisis de Big Data.En muchos escenarios de negocio, como el comercio electrónico (similar a mi proyecto de batch processing ), acumulamos grandes volúmenes de datos transaccionales. Sin embargo, a menudo carecemos de etiquetas predefinidas que nos digan "este es un cliente de alto valor" o "este es un cliente en riesgo".

En lugar de predecir un valor conocido (Clasificación), el objetivo aquí es aplicar **aprendizaje no supervisado** para **descubrir patrones y estructuras ocultas** en los datos.

El algoritmo **K-Means** es la herramienta ideal para este fin, ya que nos permite particionar los datos en *K* clústeres distintos, agrupando a los clientes en función de la similitud de sus características (como sus ingresos, edad o puntuación de gasto).El resultado es la creación de "segmentos" de clientes que la empresa puede usar para tomar decisiones informadas sobre marketing, inventario y estrategias de personalización, una aplicación que se mencionó en la Sesión 22.

## Dataset Description (20 points)

Para este proyecto de clustering, he seleccionado el dataset "Mall Customer Segmentation", obtenido de Kaggle.
https://www.kaggle.com/datasets/vjchoudhary7/customer-segmentation-tutorial-in-python

Este dataset contiene información demográfica y de comportamiento de 200 clientes de un centro comercial.

### Tamaño del Dataset
El dataset es compacto, lo que lo hace ideal para el prototipado y la validación del modelo K-Means.

Total de filas (clientes): 200

Total de columnas (atributos): 5

### Dimensiones (Clustering)
De las 5 columnas disponibles (CustomerID, Gender, Age, Annual Income (k$), Spending Score (1-100)), no todas son relevantes como dimensiones directas para el algoritmo K-Means.

CustomerID es un identificador único y será excluido del análisis.

Gender es una variable categórica que podría usarse, pero para este análisis nos centraremos en las métricas de comportamiento.

Las dimensiones o features principales que se utilizarán para segmentar a los clientes son las 3 columnas numéricas:

Age (Edad del cliente).

Annual Income (k$) (Ingreso anual del cliente en miles de dólares).

Spending Score (1-100) (Puntuación asignada por el centro comercial basada en el comportamiento de gasto).

In [2]:
# Definir Esquema
schema_columns = [
    ("CustomerID", "int"),
    ("Gender", "string"),
    ("Age", "int"),
    ("Annual Income (k$)", "int"),
    ("Spending Score (1-100)", "int")
]
customers_schema = SparkUtils.generate_schema(schema_columns)

# Cargar el Dataset
DATA_PATH = "/opt/spark/work-dir/data/ml/proyectopepe/Mall_Customers.csv"
df_customers = spark.read \
    .option("header", "true") \
    .schema(customers_schema) \
    .csv(DATA_PATH)

print("--- Esquema del Dataset ---")
df_customers.printSchema()

print(f"\n--- Tamaño del Dataset ---")
print(f"Total de filas: {df_customers.count()}")
print(f"Total de columnas: {len(df_customers.columns)}")

df_cleaned = df_customers \
    .withColumnRenamed("Annual Income (k$)", "Annual_Income") \
    .withColumnRenamed("Spending Score (1-100)", "Spending_Score")

--- Esquema del Dataset ---
root
 |-- CustomerID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Annual Income (k$): integer (nullable = true)
 |-- Spending Score (1-100): integer (nullable = true)


--- Tamaño del Dataset ---
Total de filas: 200
Total de columnas: 5


### ML Training process (30 points)

Antes de entrenar, debemos unificar nuestras dimensiones (Age, Annual_Income, Spending_Score) en una sola columna vectorial.

Herramienta: VectorAssembler.

Justificación: Los algoritmos de spark.ml están diseñados para operar sobre una única columna de tipo Vector, en lugar de múltiples columnas numéricas. Esta transformación es un paso de configuración obligatorio.

#### Hiperparámetros del Modelo (KMeans)

A continuación, se describen y justifican los hiperparámetros clave utilizados para configurar el modelo K-Means:

##### **k:**

- **Descripción:** Este es el hiperparámetro más importante de K-Means. Define cuántos segmentos de clientes intentará encontrar el algoritmo.

- **Justificación:**  
  En lugar de seleccionar un valor arbitrario para `k`, implementaremos un proceso de sintonización (similar al Lab 15).  
  Se probará un rango de valores para `k` (por ejemplo, de 2 a 10). Para cada valor, entrenaremos un modelo y calcularemos su **Silhouette Score**.  
  El valor de `k` que maximice este puntaje (el valor más cercano a 1) será seleccionado como el óptimo, ya que representa la mejor segmentación (alta cohesión interna y baja similitud entre clústeres).

##### **seed:**

- **Descripción:** El algoritmo K-Means comienza seleccionando `k` centroides iniciales de forma aleatoria.

- **Justificación:**  
  Usar un `seed` (ej. `setSeed(42)`) asegura que esta aleatoriedad inicial sea la misma cada vez que se ejecuta el código.  
  Esto es fundamental para la reproducibilidad del experimento, garantizando que obtengamos los mismos resultados y el mismo `k` óptimo en cada ejecución.

##### **featuresCol:**

- **Descripción:** Indica al modelo qué columna contiene el vector de características.

- **Justificación:**  
  Se establece en `"features"`, que es el nombre de la columna de salida (`outputCol`) que definimos en nuestro `VectorAssembler`.

In [3]:
# Preparación de Datos
feature_cols = ["Age", "Annual_Income", "Spending_Score"]

vector_assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features" # Usamos el nombre por defecto
)

assembled_df = vector_assembler.transform(df_cleaned)

print("--- Datos listos para ML (con columna 'features') ---")
assembled_df.select("features").show(5, truncate=False)

print("\n--- Buscando el k óptimo usando Silhouette Score ---")
k_values = range(2, 11)
silhouette_scores = {}

# Definir evaluador 
evaluator = ClusteringEvaluator()

for k in k_values:
    kmeans = KMeans().setK(k).setSeed(42)
    model = kmeans.fit(assembled_df)
    
    # Realizar predicciones
    predictions = model.transform(assembled_df)
    
    # Evaluar con Silhouette Score
    silhouette = evaluator.evaluate(predictions)
    silhouette_scores[k] = silhouette
    
    print(f"k = {k}, Silhouette Score: {silhouette:.4f}")

--- Datos listos para ML (con columna 'features') ---
+----------------+
|features        |
+----------------+
|[19.0,15.0,39.0]|
|[21.0,15.0,81.0]|
|[20.0,16.0,6.0] |
|[23.0,16.0,77.0]|
|[31.0,17.0,40.0]|
+----------------+
only showing top 5 rows

--- Buscando el k óptimo usando Silhouette Score ---


25/11/11 04:38:52 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


k = 2, Silhouette Score: 0.4601
k = 3, Silhouette Score: 0.4681
k = 4, Silhouette Score: 0.5732
k = 5, Silhouette Score: 0.6317
k = 6, Silhouette Score: 0.6412
k = 7, Silhouette Score: 0.6273
k = 8, Silhouette Score: 0.6262
k = 9, Silhouette Score: 0.5827
k = 10, Silhouette Score: 0.5704


In [4]:
optimal_k = max(silhouette_scores, key=silhouette_scores.get)
best_score = silhouette_scores[optimal_k]

print(f"\n--- Resultado de la Sintonización ---")
print(f"El valor óptimo de k es: {optimal_k} (Silhouette Score: {best_score:.4f})")


--- Resultado de la Sintonización ---
El valor óptimo de k es: 6 (Silhouette Score: 0.6412)


### Persistencia del Modelo

Una vez que el bucle de sintonización identifica el `optimal_k` (el `k` con el mayor Silhouette Score), se entrenará un modelo final usando ese valor `k`.  

In [5]:
MODEL_PATH = "/opt/spark/work-dir/data/mlmodels/final_project/kmeans_customer_segmentation"

print(f"\nEntrenando modelo final con k={optimal_k}...")
kmeans_final = KMeans().setK(optimal_k).setSeed(42)
final_model = kmeans_final.fit(assembled_df)

# Guardar el modelo entrenado
final_model.write().overwrite().save(MODEL_PATH)

print(f"Modelo final guardado en: {MODEL_PATH}")


Entrenando modelo final con k=6...


Modelo final guardado en: /opt/spark/work-dir/data/mlmodels/final_project/kmeans_customer_segmentation


### ML Evaluation (20 points)

Para un modelo de Clustering como K-Means, donde no tenemos "etiquetas de la verdad" (ground truth labels), la evaluación no puede basarse en métricas como **Accuracy** o **F1-Score**. En su lugar, debemos usar métricas internas que midan la calidad de la estructura del clúster.

La métrica seleccionada para evaluar el rendimiento de nuestro modelo es el **Silhouette Score**, que mide qué tan bien están definidos los clústeres. Calcula, para cada punto, qué tan similar es a su propio clúster (cohesión) en comparación con qué tan similar es a los clústeres vecinos (separación).

- **¿Cómo se interpreta?** El puntaje varía de **-1 a 1**:
  - **Cercano a 1:** Indica clústeres densos y bien separados. Este es el ideal.
  - **Cercano a 0:** Indica clústeres que se superponen o que los puntos están muy cerca del límite de decisión.
  - **Cercano a -1:** Indica que los puntos probablemente fueron asignados al clúster incorrecto.

Este es el mismo puntaje que usamos en la sección anterior para encontrar el `optimal_k`. Ahora lo usaremos para validar el rendimiento de nuestro modelo final persistido.
El proceso es el siguiente:

1. **Cargar el Modelo:**  
   Se carga el modelo K-Means final que fue guardado en el disco en la sección anterior.

2. **Cargar Datos:**  
   Se reutiliza el DataFrame `assembled_df` que ya contiene la columna `features` vectorizada.

3. **Obtener Predicciones:**  
   Se utiliza el modelo cargado para llamar a `.transform(assembled_df)`.  
   Esto añade una nueva columna al DataFrame, `prediction`, que contiene el ID del clúster (de `0` a `k-1`) al que fue asignado cada cliente.

4. **Evaluar:**  
   Se instancia `ClusteringEvaluator` configurado para la métrica **silhouette** y se aplica al DataFrame de predicciones.


In [6]:
from pyspark.ml.clustering import KMeansModel

# Cargar el Modelo
loaded_model = KMeansModel.load(MODEL_PATH)

print(f"Modelo K-Means (k={loaded_model.getK()}) cargado")

Modelo K-Means (k=6) cargado


In [7]:
# Obtener Predicciones
predictions = loaded_model.transform(assembled_df)

print("\n--- Muestra de Predicciones (Cliente -> Clúster) ---")
predictions.select("Age", "Annual_Income", "Spending_Score", "prediction").show(10)


--- Muestra de Predicciones (Cliente -> Clúster) ---
+---+-------------+--------------+----------+
|Age|Annual_Income|Spending_Score|prediction|
+---+-------------+--------------+----------+
| 19|           15|            39|         2|
| 21|           15|            81|         3|
| 20|           16|             6|         2|
| 23|           16|            77|         3|
| 31|           17|            40|         2|
| 22|           17|            76|         3|
| 35|           18|             6|         2|
| 23|           18|            94|         3|
| 64|           19|             3|         2|
| 30|           19|            72|         3|
+---+-------------+--------------+----------+
only showing top 10 rows


In [8]:
# Evaluar el Modelo
final_silhouette_score = evaluator.evaluate(predictions)

print("\n--- Evaluación Final del Modelo ---")
print(f"Silhouette Score del modelo final: {final_silhouette_score:.4f}")


--- Evaluación Final del Modelo ---
Silhouette Score del modelo final: 0.6412


In [9]:
# Análisis de Resultados
print("\n--- Conteo de Clientes por Segmento ---")
segment_summary = predictions.groupBy("prediction").count().orderBy("prediction")
segment_summary.show()

print("\n--- Centroides (Promedios) por Segmento ---")
centers = loaded_model.clusterCenters()
for i, center in enumerate(centers):
    print(f"Segmento {i} [Centroide]:")
    print(f"  -> Edad Promedio: {center[0]:.1f}")
    print(f"  -> Ingreso Anual Promedio (k$): {center[1]:.1f}")
    print(f"  -> Puntuación de Gasto Promedio: {center[2]:.1f}\n")


--- Conteo de Clientes por Segmento ---
+----------+-----+
|prediction|count|
+----------+-----+
|         0|   39|
|         1|   44|
|         2|   22|
|         3|   22|
|         4|   39|
|         5|   34|
+----------+-----+


--- Centroides (Promedios) por Segmento ---
Segmento 0 [Centroide]:
  -> Edad Promedio: 32.7
  -> Ingreso Anual Promedio (k$): 86.5
  -> Puntuación de Gasto Promedio: 82.1

Segmento 1 [Centroide]:
  -> Edad Promedio: 56.3
  -> Ingreso Anual Promedio (k$): 53.7
  -> Puntuación de Gasto Promedio: 49.4

Segmento 2 [Centroide]:
  -> Edad Promedio: 44.3
  -> Ingreso Anual Promedio (k$): 25.8
  -> Puntuación de Gasto Promedio: 20.3

Segmento 3 [Centroide]:
  -> Edad Promedio: 25.3
  -> Ingreso Anual Promedio (k$): 25.7
  -> Puntuación de Gasto Promedio: 79.4

Segmento 4 [Centroide]:
  -> Edad Promedio: 27.4
  -> Ingreso Anual Promedio (k$): 57.0
  -> Puntuación de Gasto Promedio: 48.8

Segmento 5 [Centroide]:
  -> Edad Promedio: 41.6
  -> Ingreso Anual Promedio (

In [10]:
sc.stop()
spark.stop()